In [1]:
import os

In [2]:
%pwd

'd:\\CDAC\\Machine Learning\\sentiment-analyzer\\notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\CDAC\\Machine Learning\\sentiment-analyzer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_transformed_dir: Path
    model_dir: Path
    evaluation_saved: Path

In [8]:
from sentimentAnalyzer.constant import *
from sentimentAnalyzer.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = Path(config.root_dir),
            test_transformed_dir = Path(config.test_transformed_dir),
            model_dir = Path(config.model_dir),
            evaluation_saved = Path(config.evaluation_saved)
        )

        return model_evaluation_config

In [16]:
import joblib
import pandas as pd
from sentimentAnalyzer.utils.common import load_transformed_data_file, DataInfo
from sentimentAnalyzer.logging import logger
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [17]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate(self):
        X_test, y_test = load_transformed_data_file(path=self.config.test_transformed_dir, 
                                                    data_info=DataInfo.TESTING)
        
        # Loading Models
        lrmodel = joblib.load(self.config.model_dir)
        logger.info(f"Trained Model imported from {self.config.model_dir}")

        # Classification Report
        y_pred = lrmodel.predict(X_test)
        print(f">>>>>>> Classifiaction Report <<<<<<< \n{classification_report(y_test, y_pred)}")

        # Saving Evaluation Result
        metrices = {
            "Accuracy": accuracy_score(y_test, y_pred),
            "Precision": precision_score(y_test, y_pred),
            "Recall": recall_score(y_test, y_pred),
            "F1 Score": f1_score(y_test, y_pred),
        }

        # Save metrics to a CSV
        metrics_df = pd.DataFrame(list(metrices.items()), columns=["Metric", "Value"])
        metrics_df.to_csv(self.config.evaluation_saved, index=False)
        logger.info(f"Model Performance Metrices stored at {self.config.evaluation_saved}")


In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2025-01-04 21:16:43,869: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-04 21:16:43,872: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-04 21:16:43,874: INFO: common: created directory at: artifacts]
[2025-01-04 21:16:43,875: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-04 21:16:44,125: INFO: common: Input Transformed Testing Data has been load from artifacts\data_transformation\test\X_test.npz]
[2025-01-04 21:16:44,129: INFO: common: Output Transformed Testing Data is load from artifacts\data_transformation\test\y_test.npy]
[2025-01-04 21:16:44,154: INFO: 3954773557: Trained Model imported from artifacts\train_model\trained_model.pkl]
>>>>>>> Classifiaction Report <<<<<<< 
              precision    recall  f1-score   support

           0       0.80      0.78      0.79    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.80    320000
   macro avg       0.80